# Todo (Issue)
- answer code 누적합 column 제작
- 그래프 데이터 만들기
- 해당 시험을 몇 번째 푸는지 여부 (11/21 완)
   - 해당 시험을 몇 번째 푸는지 여부 (11/21 완)
   - 시험 -> 문제, 태그로 변환도 가능하며 이전에 맞췄는지 여부도 포함 (11/21 완)
- 문제를 푸는데 걸린시간이 평균 시간보다 오래걸렸는지 여부 추가
   - 한 유저가 어떤 문제를 풀때 그 문제를 푸는데 걸린 시간이 그 문제의 평균 시간보다 적게 걸렸는지 혹은 오래 걸렸는지 여부를 column에 추가
   - 해당 문항을 맞춘 학생의 평균 시간과 틀린 학생의 평균 시간 2가지를 Feature로 주어 활용하는 방안도 있음
- 사용자의 정답률 column 제작 (11/21 완)
   - 누적 정답률, 최근 정답률 두 가지를 제작한다. 최근 정답률은 window size를 줘서 최근 N개의 문항에 대한 평균을 구할 수 있도록 (11/21 완, N개 문항은 모델 부분에서 적용)
- 문항, 시험지, 태그의 평균 정답률 (11/21 완)
   - [EDA] test에서 정답을 예측해야 되는 문제가 train에도 있는지 확인
- 문제를 푼 사용자와 사용자가 푼 문항을 MF나 SVD를 통해 user-item matrix 생성해 feature로 사용

# Todo (추가)
- 장원준 EDA (11/21 완)
   - 각 문항 번호 column 추가
   - 각 유저별 푼 카테고리 목록 추가
- 정준환 EDA
   - timestamp이용해 걸린 시간 계산
   - 오류 시간은 -1 처리, threshold 걸기(200초)
- 이수경 EDA
   - timestamp이용해 걸린 시간 계산(elapsed_time)
   - 한 유저가 동일한 시험지를 두 번 이상 풀었을 경우를 고려 못 해줌 => 고려 필요
   - 한 유저가 동일한 시험지를 연속해서 풀었을 수도 있음 => 확인 필요 => 288번 있음
   - 문제 번호는 랜덤이라는 것 주의해서 처리
   - 시간 이상치 처리(threshold 200초, 평균 처리 등)
   - 7863번 knowledgetag 제외 모두 고유한 category를 가진다는 것 유념
- 류명현 EDA
   - 시간대별 정답률 다르므로 timestamp를 나누자



In [210]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

In [211]:
train = pd.read_csv('data/train_data.csv')
test = pd.read_csv('data/test_data.csv')
submission = pd.read_csv('data/sample_submission.csv')

In [212]:
train
new_train = train.copy()
new_train

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225
...,...,...,...,...,...,...
2266581,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438
2266582,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836
2266583,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836
2266584,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836


# 기본 전처리
- 새로운 column 추가
   - question_number: 문제가 테스트 상에서 몇 번인지
   - test_cat: 시험의 대분류 category가 몇인지
   - test_id: 시험지 고유의 번호
   - test_month: Timestamp에 찍힌 달
   - test_day: Timestamp에 찍힌 일
   - test_hour: Timestamp에 찍힌 시간
- 날짜 데이터 변경
   - 2019년 12월 31일 데이터에 대해 2020년 1월 1일로 변경 후 test_year는 버림

In [213]:
new_train['question_number'] = new_train['assessmentItemID'].apply(lambda x: x[8:10]).map(int) # 문제 순서 번호
new_train['test_cat'] = new_train['testId'].apply(lambda x: x[2]) # 시험지 대분류
new_train['test_id'] = new_train['assessmentItemID'].apply(lambda x: x[4:7]) # 시험지 고유 번호
new_train["Timestamp"] = new_train["Timestamp"].astype("str") # timestamp 처리

In [214]:
new_train['test_year'] = new_train['Timestamp'].apply(lambda x: x[:4])
new_train['test_day'] = new_train['Timestamp'].apply(lambda x: x[8:10])
new_train['test_hour'] = new_train['Timestamp'].apply(lambda x: x[11:13])
new_train['test_month'] = new_train['Timestamp'].apply(lambda x: x[5:7])

In [215]:
# timestamp에서 2019년 데이터는 없앰
new_train.loc[new_train['Timestamp'].str.contains('2019'), 'test_year'] = '2020'
new_train.loc[new_train['Timestamp'].str.contains('2019'), 'test_month'] = '01'
new_train.loc[new_train['Timestamp'].str.contains('2019'), 'test_day'] = '01'

new_train[new_train['Timestamp'].str.contains('2019')]

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,question_number,test_cat,test_id,test_year,test_day,test_hour,test_month
69406,92,A020036001,A020000036,1,2019-12-31 23:17:24,7696,1,2,036,2020,01,23,01
69407,92,A020036002,A020000036,1,2019-12-31 23:17:50,7696,2,2,036,2020,01,23,01
69408,92,A020036003,A020000036,1,2019-12-31 23:18:12,7696,3,2,036,2020,01,23,01
69409,92,A020036004,A020000036,1,2019-12-31 23:18:53,7696,4,2,036,2020,01,23,01
69410,92,A020036005,A020000036,1,2019-12-31 23:20:40,7696,5,2,036,2020,01,23,01
69411,92,A020036007,A020000036,1,2019-12-31 23:22:15,7696,7,2,036,2020,01,23,01
69412,92,A020036008,A020000036,1,2019-12-31 23:23:43,7696,8,2,036,2020,01,23,01
69413,92,A020036009,A020000036,0,2019-12-31 23:24:41,7696,9,2,036,2020,01,23,01
555508,784,A060014001,A060000014,1,2019-12-31 15:08:01,7229,1,6,014,2020,01,15,01
555509,784,A060014002,A060000014,1,2019-12-31 15:08:21,7229,2,6,014,2020,01,15,01


In [216]:
new_train = new_train.drop(['test_year'], axis = 1)
new_train

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,question_number,test_cat,test_id,test_day,test_hour,test_month
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,1,6,001,24,00,03
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,2,6,001,24,00,03
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,3,6,001,24,00,03
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,4,6,001,24,00,03
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,5,6,001,24,00,03
...,...,...,...,...,...,...,...,...,...,...,...,...
2266581,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438,5,3,071,05,06,06
2266582,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836,1,4,165,21,01,08
2266583,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836,2,4,165,21,01,08
2266584,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836,3,4,165,21,01,08


# [전처리] 문항, 시험지, 태그의 평균 정답률#14

- new_train2 생성

In [217]:
# 문항 시험지 태그의 평균정답률
# 시험지의 평균 정답률
test_acc = new_train.groupby(['test_cat', 'test_id'])['answerCode'].mean().reset_index()
test_acc.columns = ['test_cat','test_id','test_acc']
new_train2 = pd.merge(left = new_train, right = test_acc, on = ['test_cat', 'test_id'], how = 'left')

# 문항의 평균 정답률
question_acc = new_train.groupby(['test_cat', 'test_id', 'question_number'])['answerCode'].mean().reset_index()
question_acc.columns = ['test_cat', 'test_id', 'question_number', 'question_acc']
new_train2 = pd.merge(left = new_train2, right = question_acc, on = ['test_cat', 'test_id', 'question_number'], how = 'left')

# 태그의 평균 정답률
tag_acc = new_train.groupby(['KnowledgeTag'])['answerCode'].mean().reset_index()
tag_acc.columns = ['KnowledgeTag', 'tag_acc']
new_train2 = pd.merge(left = new_train2, right = tag_acc, on = ['KnowledgeTag'], how = 'left')

In [218]:
new_train2

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,question_number,test_cat,test_id,test_day,test_hour,test_month,test_acc,question_acc,tag_acc
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,1,6,001,24,00,03,0.947683,0.982063,0.955022
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,2,6,001,24,00,03,0.947683,0.964126,0.913187
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,3,6,001,24,00,03,0.947683,0.910314,0.913187
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,4,6,001,24,00,03,0.947683,0.968610,0.913187
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,5,6,001,24,00,03,0.947683,0.941704,0.913187
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2266581,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438,5,3,071,05,06,06,0.662590,0.435252,0.689706
2266582,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836,1,4,165,21,01,08,0.655109,0.649635,0.697874
2266583,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836,2,4,165,21,01,08,0.655109,0.635036,0.697874
2266584,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836,3,4,165,21,01,08,0.655109,0.791971,0.697874


In [219]:
# 시간별 정답률
time_acc = new_train.groupby(['test_hour'])['answerCode'].mean().reset_index()
time_acc.columns = ['test_hour', 'time_acc']
new_train2 = pd.merge(left = new_train2, right = time_acc, on = ['test_hour'], how = 'left')

In [220]:
# 월별 정답률
month_acc = new_train.groupby(['test_month'])['answerCode'].mean().reset_index()
month_acc.columns = ['test_month', 'month_acc']
new_train2 = pd.merge(left = new_train2, right = month_acc, on = ['test_month'], how = 'left')

In [221]:
# 누적 정답률
user_acc = new_train.groupby('userID')['answerCode'].mean().reset_index()
user_acc.columns = ['userID', 'user_acc']
new_train2 = pd.merge(left = new_train2, right = user_acc, on = 'userID', how = 'left')

In [222]:
new_train2

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,question_number,test_cat,test_id,test_day,test_hour,test_month,test_acc,question_acc,tag_acc,time_acc,month_acc,user_acc
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,1,6,001,24,00,03,0.947683,0.982063,0.955022,0.649446,0.682643,0.630872
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,2,6,001,24,00,03,0.947683,0.964126,0.913187,0.649446,0.682643,0.630872
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,3,6,001,24,00,03,0.947683,0.910314,0.913187,0.649446,0.682643,0.630872
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,4,6,001,24,00,03,0.947683,0.968610,0.913187,0.649446,0.682643,0.630872
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,5,6,001,24,00,03,0.947683,0.941704,0.913187,0.649446,0.682643,0.630872
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2266581,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438,5,3,071,05,06,06,0.662590,0.435252,0.689706,0.629941,0.651967,0.555556
2266582,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836,1,4,165,21,01,08,0.655109,0.649635,0.697874,0.644321,0.657599,0.555556
2266583,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836,2,4,165,21,01,08,0.655109,0.635036,0.697874,0.644321,0.657599,0.555556
2266584,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836,3,4,165,21,01,08,0.655109,0.791971,0.697874,0.644321,0.657599,0.555556


In [223]:
new_train3 = new_train2.copy()

# [전처리] 해당 시험/문제/태그를 몇 번째 푸는지 여부

In [224]:
# 문제를 몇 번 푸는지?
exp_question = new_train3.groupby(['test_cat', 'test_id', 'question_number'])['userID'].value_counts().reset_index(name='exp_question')
new_train3 = pd.merge(left = new_train3, right = exp_question, on = ['userID', 'test_cat', 'test_id', 'question_number'], how = 'left')

In [225]:
new_train3.head(10)

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,question_number,test_cat,test_id,test_day,test_hour,test_month,test_acc,question_acc,tag_acc,time_acc,month_acc,user_acc,exp_question
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,1,6,001,24,00,03,0.947683,0.982063,0.955022,0.649446,0.682643,0.630872,1
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,2,6,001,24,00,03,0.947683,0.964126,0.913187,0.649446,0.682643,0.630872,1
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,3,6,001,24,00,03,0.947683,0.910314,0.913187,0.649446,0.682643,0.630872,1
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,4,6,001,24,00,03,0.947683,0.968610,0.913187,0.649446,0.682643,0.630872,1
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,5,6,001,24,00,03,0.947683,0.941704,0.913187,0.649446,0.682643,0.630872,1
5,0,A060001007,A060000001,1,2020-03-24 00:17:47,7225,7,6,001,24,00,03,0.947683,0.919283,0.913187,0.649446,0.682643,0.630872,1
6,0,A060003001,A060000003,0,2020-03-26 05:52:03,7226,1,6,003,26,05,03,0.790562,0.882353,0.799552,0.624776,0.682643,0.630872,1
7,0,A060003002,A060000003,1,2020-03-26 05:52:10,7226,2,6,003,26,05,03,0.790562,0.918552,0.799552,0.624776,0.682643,0.630872,1
8,0,A060003003,A060000003,1,2020-03-26 05:53:14,7226,3,6,003,26,05,03,0.790562,0.520362,0.799552,0.624776,0.682643,0.630872,1
9,0,A060003004,A060000003,1,2020-03-26 05:53:29,7226,4,6,003,26,05,03,0.790562,0.823529,0.799552,0.624776,0.682643,0.630872,1


In [226]:

# 시험을 몇 번 푸는지?

fff = new_train3.groupby(['test_cat', 'test_id'])['question_number'].unique().reset_index()
fff['len'] = fff['question_number'].apply(len)

# new_train3['len'] = new_train3['question_number'].apply(len)
# test2len = { key:value for key, value in zip(new_train3['testId'], new_train3['len'])}
# train['question_len'] = train['testId'].map(test2len)

exp_test = new_train3.groupby(['test_cat', 'test_id'])['userID'].value_counts().reset_index(name='exp_test')
new_train3 = pd.merge(left = new_train3, right = exp_test, on = ['userID', 'test_cat', 'test_id'], how = 'left')
new_train3 = pd.merge(left = new_train3, right = fff, on = ['test_cat', 'test_id'], how = 'left')
new_train3


,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,question_number_x,test_cat,test_id,test_day,test_hour,test_month,test_acc,question_acc,tag_acc,time_acc,month_acc,user_acc,exp_question,exp_test,question_number_y,len
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,1,6,001,24,00,03,0.947683,0.982063,0.955022,0.649446,0.682643,0.630872,1,6,"[1, 2, 3, 4, 5, 7]",6
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,2,6,001,24,00,03,0.947683,0.964126,0.913187,0.649446,0.682643,0.630872,1,6,"[1, 2, 3, 4, 5, 7]",6
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,3,6,001,24,00,03,0.947683,0.910314,0.913187,0.649446,0.682643,0.630872,1,6,"[1, 2, 3, 4, 5, 7]",6
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,4,6,001,24,00,03,0.947683,0.968610,0.913187,0.649446,0.682643,0.630872,1,6,"[1, 2, 3, 4, 5, 7]",6
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,5,6,001,24,00,03,0.947683,0.941704,0.913187,0.649446,0.682643,0.630872,1,6,"[1, 2, 3, 4, 5, 7]",6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2266581,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438,5,3,071,05,06,06,0.662590,0.435252,0.689706,0.629941,0.651967,0.555556,1,5,"[1, 2, 3, 4, 5]",5
2266582,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836,1,4,165,21,01,08,0.655109,0.649635,0.697874,0.644321,0.657599,0.555556,1,4,"[1, 2, 3, 4]",4
2266583,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836,2,4,165,21,01,08,0.655109,0.635036,0.697874,0.644321,0.657599,0.555556,1,4,"[1, 2, 3, 4]",4
2266584,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836,3,4,165,21,01,08,0.655109,0.791971,0.697874,0.644321,0.657599,0.555556,1,4,"[1, 2, 3, 4]",4


In [227]:
new_train3

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,question_number_x,test_cat,test_id,test_day,test_hour,test_month,test_acc,question_acc,tag_acc,time_acc,month_acc,user_acc,exp_question,exp_test,question_number_y,len
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,1,6,001,24,00,03,0.947683,0.982063,0.955022,0.649446,0.682643,0.630872,1,6,"[1, 2, 3, 4, 5, 7]",6
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,2,6,001,24,00,03,0.947683,0.964126,0.913187,0.649446,0.682643,0.630872,1,6,"[1, 2, 3, 4, 5, 7]",6
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,3,6,001,24,00,03,0.947683,0.910314,0.913187,0.649446,0.682643,0.630872,1,6,"[1, 2, 3, 4, 5, 7]",6
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,4,6,001,24,00,03,0.947683,0.968610,0.913187,0.649446,0.682643,0.630872,1,6,"[1, 2, 3, 4, 5, 7]",6
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,5,6,001,24,00,03,0.947683,0.941704,0.913187,0.649446,0.682643,0.630872,1,6,"[1, 2, 3, 4, 5, 7]",6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2266581,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438,5,3,071,05,06,06,0.662590,0.435252,0.689706,0.629941,0.651967,0.555556,1,5,"[1, 2, 3, 4, 5]",5
2266582,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836,1,4,165,21,01,08,0.655109,0.649635,0.697874,0.644321,0.657599,0.555556,1,4,"[1, 2, 3, 4]",4
2266583,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836,2,4,165,21,01,08,0.655109,0.635036,0.697874,0.644321,0.657599,0.555556,1,4,"[1, 2, 3, 4]",4
2266584,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836,3,4,165,21,01,08,0.655109,0.791971,0.697874,0.644321,0.657599,0.555556,1,4,"[1, 2, 3, 4]",4


In [228]:
type(new_train3['len'][0])

numpy.int64

In [229]:
new_train3['exp_test'] = new_train3.apply(lambda x: x['exp_test'] / x['len'], axis = 1)
# df['column3'] = df.apply(lambda x: cos_sim(x['column1'], x['column2']), axis=1)
new_train3

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,question_number_x,test_cat,test_id,test_day,test_hour,test_month,test_acc,question_acc,tag_acc,time_acc,month_acc,user_acc,exp_question,exp_test,question_number_y,len
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,1,6,001,24,00,03,0.947683,0.982063,0.955022,0.649446,0.682643,0.630872,1,1.0,"[1, 2, 3, 4, 5, 7]",6
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,2,6,001,24,00,03,0.947683,0.964126,0.913187,0.649446,0.682643,0.630872,1,1.0,"[1, 2, 3, 4, 5, 7]",6
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,3,6,001,24,00,03,0.947683,0.910314,0.913187,0.649446,0.682643,0.630872,1,1.0,"[1, 2, 3, 4, 5, 7]",6
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,4,6,001,24,00,03,0.947683,0.968610,0.913187,0.649446,0.682643,0.630872,1,1.0,"[1, 2, 3, 4, 5, 7]",6
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,5,6,001,24,00,03,0.947683,0.941704,0.913187,0.649446,0.682643,0.630872,1,1.0,"[1, 2, 3, 4, 5, 7]",6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2266581,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438,5,3,071,05,06,06,0.662590,0.435252,0.689706,0.629941,0.651967,0.555556,1,1.0,"[1, 2, 3, 4, 5]",5
2266582,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836,1,4,165,21,01,08,0.655109,0.649635,0.697874,0.644321,0.657599,0.555556,1,1.0,"[1, 2, 3, 4]",4
2266583,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836,2,4,165,21,01,08,0.655109,0.635036,0.697874,0.644321,0.657599,0.555556,1,1.0,"[1, 2, 3, 4]",4
2266584,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836,3,4,165,21,01,08,0.655109,0.791971,0.697874,0.644321,0.657599,0.555556,1,1.0,"[1, 2, 3, 4]",4


In [230]:
new_train3.rename(columns = {'question_number_x':'question_number','question_number_y':'question_nums', 'len':'question_numslen'},inplace=True)

In [231]:
new_train3['exp_test'] = new_train3['exp_test'].astype(int)

In [232]:
new_train3

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,question_number,test_cat,test_id,test_day,test_hour,test_month,test_acc,question_acc,tag_acc,time_acc,month_acc,user_acc,exp_question,exp_test,question_nums,question_numslen
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,1,6,001,24,00,03,0.947683,0.982063,0.955022,0.649446,0.682643,0.630872,1,1,"[1, 2, 3, 4, 5, 7]",6
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,2,6,001,24,00,03,0.947683,0.964126,0.913187,0.649446,0.682643,0.630872,1,1,"[1, 2, 3, 4, 5, 7]",6
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,3,6,001,24,00,03,0.947683,0.910314,0.913187,0.649446,0.682643,0.630872,1,1,"[1, 2, 3, 4, 5, 7]",6
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,4,6,001,24,00,03,0.947683,0.968610,0.913187,0.649446,0.682643,0.630872,1,1,"[1, 2, 3, 4, 5, 7]",6
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,5,6,001,24,00,03,0.947683,0.941704,0.913187,0.649446,0.682643,0.630872,1,1,"[1, 2, 3, 4, 5, 7]",6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2266581,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438,5,3,071,05,06,06,0.662590,0.435252,0.689706,0.629941,0.651967,0.555556,1,1,"[1, 2, 3, 4, 5]",5
2266582,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836,1,4,165,21,01,08,0.655109,0.649635,0.697874,0.644321,0.657599,0.555556,1,1,"[1, 2, 3, 4]",4
2266583,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836,2,4,165,21,01,08,0.655109,0.635036,0.697874,0.644321,0.657599,0.555556,1,1,"[1, 2, 3, 4]",4
2266584,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836,3,4,165,21,01,08,0.655109,0.791971,0.697874,0.644321,0.657599,0.555556,1,1,"[1, 2, 3, 4]",4


In [233]:
# 태그를 몇 번 푸는지?
exp_tag = new_train3.groupby(['KnowledgeTag'])['userID'].value_counts().reset_index(name='exp_tag')
new_train3 = pd.merge(left = new_train3, right = exp_tag, on = ['userID', 'KnowledgeTag'], how = 'left')
new_train3

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,question_number,test_cat,test_id,test_day,test_hour,test_month,test_acc,question_acc,tag_acc,time_acc,month_acc,user_acc,exp_question,exp_test,question_nums,question_numslen,exp_tag
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,1,6,001,24,00,03,0.947683,0.982063,0.955022,0.649446,0.682643,0.630872,1,1,"[1, 2, 3, 4, 5, 7]",6,1
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,2,6,001,24,00,03,0.947683,0.964126,0.913187,0.649446,0.682643,0.630872,1,1,"[1, 2, 3, 4, 5, 7]",6,5
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,3,6,001,24,00,03,0.947683,0.910314,0.913187,0.649446,0.682643,0.630872,1,1,"[1, 2, 3, 4, 5, 7]",6,5
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,4,6,001,24,00,03,0.947683,0.968610,0.913187,0.649446,0.682643,0.630872,1,1,"[1, 2, 3, 4, 5, 7]",6,5
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,5,6,001,24,00,03,0.947683,0.941704,0.913187,0.649446,0.682643,0.630872,1,1,"[1, 2, 3, 4, 5, 7]",6,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2266581,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438,5,3,071,05,06,06,0.662590,0.435252,0.689706,0.629941,0.651967,0.555556,1,1,"[1, 2, 3, 4, 5]",5,5
2266582,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836,1,4,165,21,01,08,0.655109,0.649635,0.697874,0.644321,0.657599,0.555556,1,1,"[1, 2, 3, 4]",4,4
2266583,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836,2,4,165,21,01,08,0.655109,0.635036,0.697874,0.644321,0.657599,0.555556,1,1,"[1, 2, 3, 4]",4,4
2266584,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836,3,4,165,21,01,08,0.655109,0.791971,0.697874,0.644321,0.657599,0.555556,1,1,"[1, 2, 3, 4]",4,4


In [234]:
new_train3.rename(columns = {'time_acc':'hour_acc'},inplace=True) # 실수 복구

In [235]:
new_train3.columns

Index(['userID', 'assessmentItemID', 'testId', 'answerCode', 'Timestamp',
       'KnowledgeTag', 'question_number', 'test_cat', 'test_id', 'test_day',
       'test_hour', 'test_month', 'test_acc', 'question_acc', 'tag_acc',
       'hour_acc', 'month_acc', 'user_acc', 'exp_question', 'exp_test',
       'question_nums', 'question_numslen', 'exp_tag'],
      dtype='object')

In [236]:
new_train3 = new_train3[['userID', 'assessmentItemID', 'testId', 'answerCode', 'Timestamp', 'KnowledgeTag', 'test_cat',
       'test_id', 'question_number', 'question_numslen', 'test_month', 'test_day', 'test_hour',
       'user_acc', 'test_acc', 'tag_acc', 'question_acc', 'month_acc',
       'hour_acc', 'exp_test', 'exp_tag', 'exp_question']]

In [237]:
new_train3

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,test_cat,test_id,question_number,question_numslen,test_month,test_day,test_hour,user_acc,test_acc,tag_acc,question_acc,month_acc,hour_acc,exp_test,exp_tag,exp_question
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,6,001,1,6,03,24,00,0.630872,0.947683,0.955022,0.982063,0.682643,0.649446,1,1,1
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,6,001,2,6,03,24,00,0.630872,0.947683,0.913187,0.964126,0.682643,0.649446,1,5,1
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,6,001,3,6,03,24,00,0.630872,0.947683,0.913187,0.910314,0.682643,0.649446,1,5,1
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,6,001,4,6,03,24,00,0.630872,0.947683,0.913187,0.968610,0.682643,0.649446,1,5,1
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,6,001,5,6,03,24,00,0.630872,0.947683,0.913187,0.941704,0.682643,0.649446,1,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2266581,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438,3,071,5,5,06,05,06,0.555556,0.662590,0.689706,0.435252,0.651967,0.629941,1,5,1
2266582,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836,4,165,1,4,08,21,01,0.555556,0.655109,0.697874,0.649635,0.657599,0.644321,1,4,1
2266583,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836,4,165,2,4,08,21,01,0.555556,0.655109,0.697874,0.635036,0.657599,0.644321,1,4,1
2266584,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836,4,165,3,4,08,21,01,0.555556,0.655109,0.697874,0.791971,0.657599,0.644321,1,4,1


In [238]:
new_train3.columns

Index(['userID', 'assessmentItemID', 'testId', 'answerCode', 'Timestamp',
       'KnowledgeTag', 'test_cat', 'test_id', 'question_number',
       'question_numslen', 'test_month', 'test_day', 'test_hour', 'user_acc',
       'test_acc', 'tag_acc', 'question_acc', 'month_acc', 'hour_acc',
       'exp_test', 'exp_tag', 'exp_question'],
      dtype='object')

# 정리
## train
- 원본
## new_train
- 다른 기법 없이 기존 columns만 정리하여 새로운 column 생성
- userID	
- answerCode	
- Timestamp	
- KnowledgeTag	
- question_number	
- test_cat	
- test_id	
- test_day	
- test_hour	
- test_month
## new_train2
- 새로운 기법(완료된 project 참조) 적용해 column 생성
- userID	
- answerCode	
- Timestamp	
- KnowledgeTag	
- question_number	
- test_cat	
- test_id	
- test_day	
- test_hour	
- test_month	
- test_acc	
- question_acc	
- tag_acc	
- time_acc	
- month_acc	
- user_acc
## new_train3 (11/21 월요일)
- 태그, 문제, 테스트 중복 횟수 column 추가~
- userID: 유저 아이디
- answerCode: 정답 여부 1: 정답 0: 오답
- Timestamp: 문제를 풀기 시작한 시간
- KnowledgeTag: 문제 유형 태그
- test_cat: 시험의 대분류
- test_id: 시험의 고유 아이디
- question_number: 시험에서의 문제 번호
   - test_cat, test_id, question_number는 기존의 assessmentItemID를 분해한 것
- question_numslen: 시험에서 몇 개의 번호가 존재하는가 (1,2,3,5가 출제되었다면 question_numslen는 4)
- test_month: 시험을 본 달
- test_day: 시험을 본 일
- test_hour: 시험을 본 시간
   - test_month, test_day, test_hour는 Timestamp에서 의미가 있다고 판단되는 feature
   - 2019/12/31이 일부 있었으나, 2020/01/01로 바꾸어서 test_year가 의미 없어짐
- user_acc: 해당 유저의 전체 정답률
- test_acc: 해당 test의 정답률
- tag_acc: 해당 tag의 정답률
- question_acc: 해당 문제의 정답률
- month_acc: 이 달의 전체 유저의 정답률
- hour_acc: 이 시간의 전체 유저의 정답률
- exp_test: 해당 테스트를 경험한 수 (최소 1 최대 3)
- exp_tag: 해당 태그를 경험한 수
- exp_question: 해당 문제를 경험한 수(대체로 exp_test를 따라갈 것)

In [239]:
new_train4 = new_train3.copy()

In [240]:
new_train4['post_Timestamp'] = new_train4.groupby(['userID', 'test_id', 'test_day'])['Timestamp'].shift(-1)

In [241]:
new_train4.columns

Index(['userID', 'assessmentItemID', 'testId', 'answerCode', 'Timestamp',
       'KnowledgeTag', 'test_cat', 'test_id', 'question_number',
       'question_numslen', 'test_month', 'test_day', 'test_hour', 'user_acc',
       'test_acc', 'tag_acc', 'question_acc', 'month_acc', 'hour_acc',
       'exp_test', 'exp_tag', 'exp_question', 'post_Timestamp'],
      dtype='object')

In [242]:
new_train4 = new_train4[['userID', 'assessmentItemID', 'testId', 'answerCode', 'Timestamp', 'post_Timestamp',
       'KnowledgeTag', 'test_cat', 'test_id', 'question_number',
       'question_numslen', 'test_month', 'test_day', 'test_hour', 'user_acc',
       'test_acc', 'tag_acc', 'question_acc', 'month_acc', 'hour_acc',
       'exp_test', 'exp_tag', 'exp_question']]

In [243]:
new_train4[new_train4['userID'] == 0][120:130]

,userID,assessmentItemID,testId,answerCode,Timestamp,post_Timestamp,KnowledgeTag,test_cat,test_id,question_number,question_numslen,test_month,test_day,test_hour,user_acc,test_acc,tag_acc,question_acc,month_acc,hour_acc,exp_test,exp_tag,exp_question
120,0,A080016005,A080000016,1,2020-05-07 00:51:29,2020-05-07 00:51:48,4669,8,016,5,8,05,07,00,0.630872,0.602778,0.468662,0.518519,0.643317,0.649446,1,5,1
121,0,A080016006,A080000016,1,2020-05-07 00:51:48,2020-05-07 00:52:47,4664,8,016,6,8,05,07,00,0.630872,0.602778,0.569902,0.688889,0.643317,0.649446,1,3,1
122,0,A080016007,A080000016,1,2020-05-07 00:52:47,2020-05-07 00:52:47,4672,8,016,7,8,05,07,00,0.630872,0.602778,0.432649,0.511111,0.643317,0.649446,1,6,1
123,0,A080016008,A080000016,0,2020-05-07 00:52:47,NaN,4672,8,016,8,8,05,07,00,0.630872,0.602778,0.432649,0.511111,0.643317,0.649446,1,6,1
124,0,A060024001,A060000024,1,2020-05-07 01:20:10,2020-05-07 01:20:16,595,6,024,1,7,05,07,01,0.630872,0.723214,0.690083,0.906250,0.643317,0.644321,1,5,1
125,0,A060024002,A060000024,1,2020-05-07 01:20:16,2020-05-07 01:20:28,594,6,024,2,7,05,07,01,0.630872,0.723214,0.774077,0.919643,0.643317,0.644321,1,2,1
126,0,A060024003,A060000024,1,2020-05-07 01:20:28,2020-05-07 01:20:36,595,6,024,3,7,05,07,01,0.630872,0.723214,0.690083,0.700893,0.643317,0.644321,1,5,1
127,0,A060024004,A060000024,1,2020-05-07 01:20:36,2020-05-07 01:20:48,595,6,024,4,7,05,07,01,0.630872,0.723214,0.690083,0.812500,0.643317,0.644321,1,5,1
128,0,A060024005,A060000024,1,2020-05-07 01:20:48,2020-05-07 01:21:06,595,6,024,5,7,05,07,01,0.630872,0.723214,0.690083,0.616071,0.643317,0.644321,1,5,1
129,0,A060024006,A060000024,1,2020-05-07 01:21:06,2020-05-07 01:21:19,594,6,024,6,7,05,07,01,0.630872,0.723214,0.774077,0.799107,0.643317,0.644321,1,2,1


In [244]:
# new_train3[new_train3['Timestamp'] == '2020-08-18 23:40:18']
# train[train['Timestamp'] == '2020-08-18 23:40:18']
# new_train4[(new_train4['userID'] == 4492 )& (new_train4['testId']=='A060000152')]

In [245]:
pd.set_option('display.max_columns', None)

#### 수경 EDA에서 Elapsed time 가져옴

In [246]:
# elapsed time: 문제 푸는데 소요 된 시간
new_train4['Timestamp'] = pd.to_datetime(new_train4['Timestamp'])
new_train4['elapsed_time'] = new_train4['Timestamp'].astype(int)

tmp = list(new_train4['elapsed_time'].diff() // 10**9)
tmp.append(99999)
new_train4['elapsed_time'] = tmp[1:]

# user 또는 시험지가 바뀔 때 값이 잘못 들어가서 그 경우를 -1로 바꿔줌
# -> 이 경우에는 user가 같은 시험지를 풀었을 때 값을 고려 못 해줌
new_train4['Timestamp'] = new_train4['Timestamp'].astype(str)
tmp_dict = new_train4.groupby(['userID', 'testId'])['Timestamp'].max().to_frame().reset_index()
tmp_dict['rev_time'] = -1
tmp_dict = tmp_dict.set_index(['userID', 'testId', 'Timestamp']).to_dict()
tmp_dict = tmp_dict['rev_time']
new_train4['tmp'] = tuple(zip(new_train4.userID, new_train4.testId, new_train4.Timestamp))
new_train4['tmp'] = new_train4['tmp'].map(tmp_dict)
new_train4['tmp'] = new_train4['tmp'].fillna(new_train4['elapsed_time'])
new_train4['elapsed_time'] = new_train4['tmp']
new_train4.drop(['tmp'], axis=1, inplace=True)

In [247]:
new_train4[(new_train4['userID'] == 4492 )& (new_train4['testId']=='A060000152')]

,userID,assessmentItemID,testId,answerCode,Timestamp,post_Timestamp,KnowledgeTag,test_cat,test_id,question_number,question_numslen,test_month,test_day,test_hour,user_acc,test_acc,tag_acc,question_acc,month_acc,hour_acc,exp_test,exp_tag,exp_question,elapsed_time
2058390,4492,A060152001,A060000152,1,2020-04-08 11:37:36,2020-04-08 11:38:08,1458,6,152,1,7,04,08,11,0.339869,0.635965,0.725819,0.846491,0.649545,0.709786,2,14,2,32.0
2058391,4492,A060152002,A060000152,0,2020-04-08 11:38:08,2020-04-08 11:38:12,1458,6,152,2,7,04,08,11,0.339869,0.635965,0.725819,0.763158,0.649545,0.709786,2,14,2,4.0
2058392,4492,A060152003,A060000152,0,2020-04-08 11:38:12,2020-04-08 11:38:16,1458,6,152,3,7,04,08,11,0.339869,0.635965,0.725819,0.736842,0.649545,0.709786,2,14,2,4.0
2058393,4492,A060152004,A060000152,0,2020-04-08 11:38:16,2020-04-08 11:38:19,1458,6,152,4,7,04,08,11,0.339869,0.635965,0.725819,0.706140,0.649545,0.709786,2,14,2,3.0
2058394,4492,A060152005,A060000152,0,2020-04-08 11:38:19,2020-04-08 11:38:22,1458,6,152,5,7,04,08,11,0.339869,0.635965,0.725819,0.385965,0.649545,0.709786,2,14,2,3.0
2058395,4492,A060152006,A060000152,0,2020-04-08 11:38:22,2020-04-08 11:38:26,1458,6,152,6,7,04,08,11,0.339869,0.635965,0.725819,0.517544,0.649545,0.709786,2,14,2,4.0
2058396,4492,A060152007,A060000152,0,2020-04-08 11:38:26,NaN,1458,6,152,7,7,04,08,11,0.339869,0.635965,0.725819,0.495614,0.649545,0.709786,2,14,2,14763424.0
2058418,4492,A060152001,A060000152,1,2020-10-07 10:29:54,2020-10-07 10:38:55,1458,6,152,1,7,10,07,10,0.339869,0.635965,0.725819,0.846491,0.670372,0.699632,2,14,2,541.0
2058419,4492,A060152002,A060000152,1,2020-10-07 10:38:55,2020-10-07 10:41:01,1458,6,152,2,7,10,07,10,0.339869,0.635965,0.725819,0.763158,0.670372,0.699632,2,14,2,126.0
2058420,4492,A060152003,A060000152,1,2020-10-07 10:41:01,2020-10-07 10:41:10,1458,6,152,3,7,10,07,10,0.339869,0.635965,0.725819,0.736842,0.670372,0.699632,2,14,2,9.0


In [251]:
# tmp = new_train4.dropna(axis=0)

In [ ]:
# new_train4['Timestamp'] = pd.to_datetime(new_train4['Timestamp'])
# new_train4['pre_Timestamp'] = pd.to_datetime(new_train4['pre_Timestamp'])

# new_train4['elapsed_time'] = new_train4.apply(lambda x: (x['pre_Timestamp'] - x['Timestamp']) if not x['pre_Timestamp'].notnull() else -1)

In [252]:
new_train4['elapsed_time'].value_counts().sort_index() # 총 369284개가 -1

-1.0           369284
 0.0             3278
 1.0            53821
 2.0           109998
 3.0            75778
                ...  
 7567850.0          1
 7682832.0          1
 8898824.0          1
 12292722.0         1
 14763424.0         1
Name: elapsed_time, Length: 14118, dtype: int64

In [253]:
# 연속해서 동일한 시험지를 푼 사람들을 -1 붙여 줌

tmp = 1
same_test = []
for i in tqdm(range(1,len(new_train4))):
    cur_test = new_train4['testId'][i]
    prev_test = new_train4['testId'][i-1]
    if cur_test == prev_test:
        tmp += 1
        if tmp > new_train4['question_numslen'][i]:
            same_test.extend([i-1])
            tmp = 1
    else:
        tmp = 1
        
new_train4.loc[same_test, 'elapsed_time'] = -1

100%|██████████| 2266585/2266585 [00:47<00:00, 47463.21it/s]


In [254]:
new_train4[(new_train4['userID'] == 4492 )& (new_train4['testId']=='A060000152')].loc[2058390]

userID                             4492
assessmentItemID             A060152001
testId                       A060000152
answerCode                            1
Timestamp           2020-04-08 11:37:36
post_Timestamp      2020-04-08 11:38:08
KnowledgeTag                       1458
test_cat                              6
test_id                             152
question_number                       1
question_numslen                      7
test_month                           04
test_day                             08
test_hour                            11
user_acc                       0.339869
test_acc                       0.635965
tag_acc                        0.725819
question_acc                   0.846491
month_acc                      0.649545
hour_acc                       0.709786
exp_test                              2
exp_tag                              14
exp_question                          2
elapsed_time                       32.0
Name: 2058390, dtype: object

In [255]:
new_train4[(new_train4['userID'] == 4492 )& (new_train4['testId']=='A060000152')]

,userID,assessmentItemID,testId,answerCode,Timestamp,post_Timestamp,KnowledgeTag,test_cat,test_id,question_number,question_numslen,test_month,test_day,test_hour,user_acc,test_acc,tag_acc,question_acc,month_acc,hour_acc,exp_test,exp_tag,exp_question,elapsed_time
2058390,4492,A060152001,A060000152,1,2020-04-08 11:37:36,2020-04-08 11:38:08,1458,6,152,1,7,04,08,11,0.339869,0.635965,0.725819,0.846491,0.649545,0.709786,2,14,2,32.0
2058391,4492,A060152002,A060000152,0,2020-04-08 11:38:08,2020-04-08 11:38:12,1458,6,152,2,7,04,08,11,0.339869,0.635965,0.725819,0.763158,0.649545,0.709786,2,14,2,4.0
2058392,4492,A060152003,A060000152,0,2020-04-08 11:38:12,2020-04-08 11:38:16,1458,6,152,3,7,04,08,11,0.339869,0.635965,0.725819,0.736842,0.649545,0.709786,2,14,2,4.0
2058393,4492,A060152004,A060000152,0,2020-04-08 11:38:16,2020-04-08 11:38:19,1458,6,152,4,7,04,08,11,0.339869,0.635965,0.725819,0.706140,0.649545,0.709786,2,14,2,3.0
2058394,4492,A060152005,A060000152,0,2020-04-08 11:38:19,2020-04-08 11:38:22,1458,6,152,5,7,04,08,11,0.339869,0.635965,0.725819,0.385965,0.649545,0.709786,2,14,2,3.0
2058395,4492,A060152006,A060000152,0,2020-04-08 11:38:22,2020-04-08 11:38:26,1458,6,152,6,7,04,08,11,0.339869,0.635965,0.725819,0.517544,0.649545,0.709786,2,14,2,4.0
2058396,4492,A060152007,A060000152,0,2020-04-08 11:38:26,NaN,1458,6,152,7,7,04,08,11,0.339869,0.635965,0.725819,0.495614,0.649545,0.709786,2,14,2,14763424.0
2058418,4492,A060152001,A060000152,1,2020-10-07 10:29:54,2020-10-07 10:38:55,1458,6,152,1,7,10,07,10,0.339869,0.635965,0.725819,0.846491,0.670372,0.699632,2,14,2,541.0
2058419,4492,A060152002,A060000152,1,2020-10-07 10:38:55,2020-10-07 10:41:01,1458,6,152,2,7,10,07,10,0.339869,0.635965,0.725819,0.763158,0.670372,0.699632,2,14,2,126.0
2058420,4492,A060152003,A060000152,1,2020-10-07 10:41:01,2020-10-07 10:41:10,1458,6,152,3,7,10,07,10,0.339869,0.635965,0.725819,0.736842,0.670372,0.699632,2,14,2,9.0


In [259]:
new_train4 = new_train4[['userID', 'assessmentItemID', 'testId', 'answerCode', 'Timestamp',
       'post_Timestamp','elapsed_time', 'KnowledgeTag', 'test_cat', 'test_id',
       'question_number', 'question_numslen', 'test_month', 'test_day',
       'test_hour', 'user_acc', 'test_acc', 'tag_acc', 'question_acc',
       'month_acc', 'hour_acc', 'exp_test', 'exp_tag', 'exp_question'
       ]]

# 여기까지 11월 22일

#### Elapsed time에 문제가 되는 것들
- 우선! 아래 문제가 되는 것들은 NaN으로 두고, 모델에서 처리하는 것을 목표로 함
- 따라서 일부 정상치가 이상치로 판단된다고 하더라도, 후에 적절히 채워 넣을 것이므로 상관 없다는 판단(데정연)
   - 유저가 바뀐다면? (처리 완료)
   - 다른 테스트를 푼다면? (처리 완료)
   - 같은 테스트를 시간 간격을 두고 푼다면? (처리완료 - day 변화로 판단 >> 물론 정상치도 일부 날라갈 듯)
   - 같은 테스트를 시간 간격 없이 푼다면? 정상치랑 구분 어케함?
   - 정상 데이터라도 딴 짓 하는 애들 때문에 엄청 큰 값이 잡힌다면? 어케 처리함?
- 사실 결국 정상치든, 이상치든 시간에 대한 문제인데 그럼 Threshold를 몇으로 설정하여 NaN을 잡는가가 문제